# PREPROCESAMIENTO DE DATOS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scr.explore_utils import null_analysis, column_type_dataframe

In [3]:
# Configuramos que se muestren todas las columnas de nuestros dataframes
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv("files\input\dataset_estudiantes.csv")

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17180\676143788.py:1: SyntaxWarning: invalid escape sequence '\i'
  df = pd.read_csv("files\input\dataset_estudiantes.csv")


In [5]:
df.rename(str.upper, axis='columns', inplace=True)

## 1. Gestión de duplicados

### Plan de acción
Según el EDA, no se detectaron valores duplicados en el dataset. Sin embargo, es importante verificar nuevamente antes de proceder con el modelado. Si se detectan duplicados, se evaluará si son registros idénticos que deben eliminarse o si representan casos legítimos (aunque poco probable en este contexto).

## 2. Gestión de valores nulos

### Plan de acción
Según el EDA, se identificaron valores faltantes en tres variables que requieren imputación:

- **HORAS_SUENO** (15% faltantes, 150 registros):
  - Estrategia recomendada: Imputación por mediana (7.02 horas) o mediante modelo predictivo (KNN Imputer, Random Forest Imputer)
  - Justificación: La distribución es relativamente normal y centrada alrededor de 7 horas, la mediana es robusta a outliers
  
- **HORARIO_ESTUDIO_PREFERIDO** (10% faltantes, 100 registros):
  - Estrategia recomendada: Imputación por moda (categoría más frecuente: "Noche") o crear categoría "Desconocido"
  - Alternativa: Modelo predictivo basado en otras variables (ej: estilo de aprendizaje, horas de estudio)
  
- **ESTILO_APRENDIZAJE** (5% faltantes, 50 registros):
  - Estrategia recomendada: Imputación por moda (categoría más frecuente: "Visual") o crear categoría "Desconocido"
  - Alternativa: Modelo predictivo

**Implementación**: Probar ambas estrategias (imputación simple vs modelo predictivo) y comparar el impacto en el rendimiento del modelo.

## 3. Gestión de valores atípicos (outliers)

### Plan de acción
Según el EDA, se detectaron outliers en las siguientes variables:

- **TASA_ASISTENCIA**: 4 outliers detectados (0.4% del total) mediante método IQR
  - Estrategia: Revisar si son errores de datos o valores válidos
  - Si son válidos: Mantener (son pocos y pueden ser informativos)
  - Si son errores: Eliminar o capar (winsorization) a los límites del IQR
  
- **HORAS_ESTUDIO_SEMANAL**: Revisar valores extremos (rango 1-25 horas)
  - Validar si valores muy bajos (<2 horas) o muy altos (>20 horas) son realistas
  - Considerar capar si son errores de medición

**Implementación**: 
1. Analizar cada outlier individualmente para determinar si es un error o un valor válido
2. Si son errores, aplicar winsorization o eliminación según corresponda
3. Documentar las decisiones tomadas para cada caso

## 4. Codificación de variables categóricas (encoding)

### Plan de acción
Según el EDA, tenemos 4 variables categóricas que requieren encoding. La estrategia dependerá de si existe un orden natural en las categorías:

- **NIVEL_DIFICULTAD** (3 categorías: Fácil, Medio, Difícil):
  - Estrategia: **Ordinal Encoding** (Fácil=0, Medio=1, Difícil=2)
  - Justificación: Existe un orden lógico y el EDA muestra diferencias graduales en el rendimiento según el nivel
  
- **TIENE_TUTOR** (2 categorías: Sí, No):
  - Estrategia: **Binary Encoding** (Sí=1, No=0)
  - Justificación: Variable binaria simple
  
- **HORARIO_ESTUDIO_PREFERIDO** (3 categorías: Mañana, Tarde, Noche):
  - Estrategia: **One-Hot Encoding** (3 columnas binarias)
  - Justificación: No hay un orden natural claro entre los horarios
  
- **ESTILO_APRENDIZAJE** (4 categorías: Visual, Auditivo, Lectura/Escritura, Kinestésico):
  - Estrategia: **One-Hot Encoding** (4 columnas binarias)
  - Justificación: No hay un orden natural entre los estilos

**Implementación**: 
- Usar `OrdinalEncoder` de scikit-learn para NIVEL_DIFICULTAD
- Usar `LabelEncoder` o mapeo manual para TIENE_TUTOR
- Usar `OneHotEncoder` o `pd.get_dummies()` para HORARIO_ESTUDIO_PREFERIDO y ESTILO_APRENDIZAJE

## 5. Estandarización de las variables numéricas

### Plan de acción
La necesidad de escalado/normalización depende del tipo de modelo que se vaya a utilizar:

**Modelos que REQUIEREN escalado:**
- Modelos basados en distancias: KNN, SVM
- Redes neuronales
- Modelos con regularización: Ridge, Lasso, Elastic Net
- Análisis de componentes principales (PCA)

**Modelos que NO requieren escalado:**
- Árboles de decisión
- Random Forest
- Gradient Boosting (XGBoost, LightGBM, CatBoost)
- Naive Bayes

**Métodos de escalado recomendados:**
- **StandardScaler** (media=0, std=1): Para distribuciones relativamente normales. Apropiado para la mayoría de nuestras variables numéricas según el EDA
- **MinMaxScaler** (rango 0-1): Si queremos mantener la interpretabilidad y el rango original
- **RobustScaler**: Si hay presencia de outliers (usa mediana e IQR en lugar de media y desviación estándar)

**Implementación**: 
- Crear pipelines que permitan activar/desactivar el escalado según el modelo
- Aplicar el escalado SOLO después de dividir train/test para evitar data leakage
- Guardar los parámetros del scaler del conjunto de entrenamiento para aplicar la misma transformación al test

## 6. Selección de variables (feature selection)

### Plan de acción
Según el EDA, se identificaron variables con baja o nula correlación con las variables objetivo:

**Variables a considerar eliminar:**
- **EDAD**: 
  - Correlación prácticamente nula (-0.01) con NOTA_FINAL
  - Sin diferencias significativas entre aprobados y suspendidos (ambos con mediana de 24 años)
  - Recomendación: Mantener inicialmente para validación, eliminar si no mejora el modelo
  
- **HORAS_SUENO**: 
  - Correlación muy débil (0.07) con NOTA_FINAL
  - Sin diferencias significativas entre aprobados (mediana: 7.01) y suspendidos (mediana: 7.05)
  - Además tiene 15% de valores faltantes
  - Recomendación: Mantener inicialmente para validación, eliminar si no mejora el modelo

**Variables a mantener** (alta predictividad):
- **HORAS_ESTUDIO_SEMANAL**: Correlación 0.51 con NOTA_FINAL, diferencia de 5.17 horas entre aprobados/suspendidos
- **NOTA_ANTERIOR**: Correlación 0.47 con NOTA_FINAL, diferencia de 12.49 puntos entre aprobados/suspendidos
- **TASA_ASISTENCIA**: Correlación 0.32 con NOTA_FINAL, diferencia de 14.44 puntos porcentuales
- **TIENE_TUTOR**: Diferencia de 10 puntos porcentuales en tasa de aprobado
- **Todas las variables categóricas**: Muestran impacto en los objetivos según el EDA

**Feature Engineering - Interacciones sugeridas:**
- `HORAS_ESTUDIO_SEMANAL × TIENE_TUTOR`: ¿El tutor multiplica el efecto de las horas de estudio?
- `NOTA_ANTERIOR × NIVEL_DIFICULTAD`: ¿La dificultad percibida modera el efecto de la nota anterior?
- `TASA_ASISTENCIA × HORAS_ESTUDIO_SEMANAL`: Efecto sinérgico de asistencia y estudio

**Implementación**: 
- Probar modelos con y sin EDAD y HORAS_SUENO para comparar rendimiento
- Crear features de interacción para las combinaciones sugeridas
- Usar técnicas de selección de features (SelectKBest, Recursive Feature Elimination) si el número de features crece significativamente

## 7. Preparación específica para REGRESIÓN (NOTA_FINAL)

### Plan de acción
Para el modelo de regresión que predice NOTA_FINAL:

**Preparación de la variable objetivo:**
- **NOTA_FINAL**: Variable continua (rango 30-100)
- El EDA muestra distribución relativamente normal (media: 71.44, mediana: 71.4, std: 9.56)
- Probablemente no necesite transformación, pero validar distribución de residuos según modelo elegido

**División Train/Test/Validation:**
- División recomendada: 70% train, 15% validation, 15% test
- Estratificar NO es necesario (variable continua)
- Asegurar que la distribución de NOTA_FINAL sea similar en todos los sets (verificar estadísticas descriptivas)

**Consideraciones adicionales:**
- No se requiere balanceo de clases (es regresión)
- Validar supuestos del modelo según el algoritmo elegido (normalidad de residuos, homocedasticidad, etc.)


## 8. Preparación específica para CLASIFICACIÓN (APROBADO)

### Plan de acción
Para el modelo de clasificación que predice APROBADO:

**Balanceo de Clases (CRÍTICO):**
- **Problema detectado**: Desbalance severo (89.8% aprobados vs 10.2% suspendidos)
- Este es el paso MÁS IMPORTANTE para clasificación, ya que sin balanceo el modelo tendrá sesgo hacia la clase mayoritaria

**Estrategias a probar y comparar:**
1. **SMOTE** (Synthetic Minority Oversampling Technique):
   - Generar muestras sintéticas de la clase minoritaria (suspendidos)
   - Ventaja: Aumenta la clase minoritaria sin perder información
   - Implementación: `imbalanced-learn` library (`SMOTE()`)

2. **Ajuste de pesos en el modelo**:
   - Usar `class_weight='balanced'` en modelos de scikit-learn
   - Ajustar manualmente pesos según ratio: {0: 8.98, 1: 1.02} (aproximado)
   - Ventaja: No modifica los datos, solo ajusta la función de costo

3. **Combinación SMOTE + Tomek Links**:
   - SMOTE para oversampling + Tomek Links para limpieza de muestras
   - Puede mejorar la calidad de las muestras sintéticas

4. **Undersampling** (menos recomendado):
   - Reducir la clase mayoritaria
   - Riesgo: Pérdida de información valiosa
   - Solo considerar si el dataset es muy grande

**Métricas de evaluación apropiadas:**
- ❌ NO usar **Accuracy** (será engañoso, siempre predecirá "Aprobado")
- ✅ Usar: **Precision**, **Recall**, **F1-score**, **ROC-AUC**, **Matriz de confusión**
- Priorizar **Recall de la clase minoritaria** (suspendidos) si es crítico detectarlos

**División Train/Test/Validation:**
- División recomendada: 70% train, 15% validation, 15% test
- **IMPORTANTE**: Estratificar por APROBADO para mantener proporciones en todos los sets
- **CRÍTICO**: Aplicar balanceo SOLO en el conjunto de entrenamiento, NO en validation/test

**Preparación de la variable objetivo:**
- **APROBADO**: Variable binaria (0=Suspenso, 1=Aprobado)
- Verificar que esté como tipo int o bool
- Ya está en formato correcto según el EDA
